# Library

In [1]:
import pandas as pd
import numpy as np

import requests

# Function

In [2]:
BASE_URL = 'https://your-url.com'  # Sample URL, replace with actual URL

def get_login_token():
    print('status: get login token')
    url_login = f'{BASE_URL}/login'
    credentials = {
        "username": "your_username",  # Sample username
        "password": "your_password"   # Sample password
    }
    response = requests.post(url_login, json=credentials)
    print('status: success get login token')
    return response.json().get('token')


def get_flight_data(token, date):
    url_flight = f'{BASE_URL}/flight'
    print(f'status: collecting data {date}')

    # Sample parameters (replace with actual parameters)
    flight_params = {
        "first-param": "your_first_param",  # Sample parameter
        "second-param": "your_second_param"  # Sample parameter
    }

    headers = {'x-access-token': token}
    response = requests.get(url_flight, headers=headers, json=flight_params)
    return response.json().get('data', [])


def prepare_dataframe_data(token, dates):
    df_data = []

    for date in dates:
        flight_response = get_flight_data(token, date)
        
        for item in flight_response:
            manifest_list = item.get('MANIFEST', [{}])  # Get MANIFEST list or a list with an empty dictionary
            if manifest_list:  # Check if the list is not empty
                manifest_data = manifest_list[0]  # Get the first dictionary from the MANIFEST list
            else:
                manifest_data = {'ADULT': 0, 'CARGO': 0, 'CHILD': 0, 'INFANT': 0}  # Fill with zeros

            # Convert values in manifest_data to integers
            manifest_data = {key: int(value) for key, value in manifest_data.items()}

            item.pop('MANIFEST', None)  # Remove the MANIFEST key from the dictionary
            item.update(manifest_data)   # Update dictionary with MANIFEST data
            df_data.append(item)
            
    print('preparing dataframe data')         
    return df_data


def create_dataframe(df_data):
    print('status: creating dataframe')
    df = pd.DataFrame(df_data)
    
    # Convert 'MANIFEST' columns to integers
    int_columns = ['CAPACITY','ADULT', 'CARGO', 'CHILD', 'INFANT']
    # Replace infinity values with NaN first
    df[int_columns] = df[int_columns].replace([np.inf, -np.inf], np.nan)

    # Now, fill NaN values with 0
    df[int_columns] = df[int_columns].fillna(0).astype(int)
    # df[int_columns] = df[int_columns].astype(int)
    
    # Convert datetime columns to datetime format
    date_columns = ['ACTUAL_TIME', 'DATE', 'SCHEDULED_TIME']
    df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, format='mixed', dayfirst=True, errors='coerce'))
    
    # Calculate 'total pax' column
    df['TOTAL_PAX'] = df['ADULT'] + df['CHILD'] + df['INFANT']
    return df

# Get Data

In [ ]:
start_date = '2023-10-01'
end_date = '2023-10-31'

dates = pd.date_range(start_date, end_date).strftime('%Y-%m-%d').tolist()

token = get_login_token()
df_data = prepare_dataframe_data(token, dates)
df = create_dataframe(df_data)

In [6]:
df.to_csv('data-oktober2023.csv')